In [15]:
from IPython.display import display, HTML,clear_output
import random as rnd
import time

In [66]:
class SVGCanvas:
    def __init__(self,max_number,size):
        self.width = max_number*size
        self.height = max_number*size
        self.size = size
        self.max_number = max_number
        self.html_s =""
    
    def clear(self):
        self.html_s = '<svg width="'+str(self.width)+'" height="'+str(self.height)+'"style="border:1px solid black">'
    
    def rect(self,x_pos,y_pos,color):
        x_pos = x_pos * self.size
        y_pos = y_pos * self.size
        str_int_x = str(int(x_pos))
        str_int_y = str(int(y_pos))
        str_int_width = str(int(self.size))
        str_int_height = str(int(self.size))
        self.html_s +='<rect x="'+str_int_x+'" y="'+str_int_y+'" width="'+str_int_width+'" height="'+str_int_height+'" fill="'+color+'"/>'

    def circle(self,x_pos,y_pos,color):
        x_pos = x_pos * self.size+(self.size/2)
        y_pos = y_pos * self.size+(self.size/2)
        str_int_x = str(int(x_pos))
        str_int_y = str(int(y_pos))
        str_int_r = str(int(self.size/2))
        self.html_s +='<circle cx="'+str_int_x+'" cy="'+str_int_y+'" r="'+str_int_r+'" fill="'+color+'"/>'

    def getCanvas(self):
        self.html_s += "</svg>"
        return HTML(self.html_s)

In [67]:
class Counter:
    def __init__(self):
        self.tnow =0
        self.tnext = 1
        
    def update(self):
        # This switches the 0 1 state of now and next
        self.tnow = (self.tnow+1) % 2
        self.tnext = (self.tnext+1) % 2

In [68]:
class Cell:
    max_resource = 30
    def __init__(self,x_pos,y_pos,svg,counter):
        self.neighbours = []
        self.number_of_neighbours=-1
        self.x_pos = x_pos
        self.y_pos = y_pos
        self.svg = svg
        self.resource =0.0
        self.counter = counter
        self.occupant = None
        
    def addNeighbour(self,cell):
        self.neighbours.append(cell)
        self.number_of_neighbours+=1
    
    def randomNeighbour(self):
        return rnd.choice(self.neighbours)
    
    
    @property
    def resource_rgb(self):
        return str(int(255-(self.resource*255)/Cell.max_resource))
    
    @property
    def empty(self):
        return self.occupant == None
    
    def draw(self):
        self.svg.rect(self.x_pos,self.y_pos,'rgb(255,'+self.resource_rgb+','+self.resource_rgb+')')
        if self.occupant !=None:
            self.svg.circle(self.x_pos,self.y_pos,'blue')
    
    def paint(self,color):
        self.svg.rect(self.x_pos,self.y_pos,color)

    def printPos(self):
        print("x_pos:",self.x_pos)
        print("y_pos:",self.y_pos)

    def print(self):
        self.printPos()
        print()
        for cell in self.neighbours:
            cell.printPos()
        print()

    def search_res(self,depth):
        max = [self.resource,0,self]
        off_set = rnd.randint(0,4)
        for i in range(4):
            c = (i+off_set)%4
            cell = self
            for j in range(depth):
                cell = cell.neighbours[c]
                if cell.empty:
                    res = cell.resource
                    if res>= max[0]:
                        max=[res,j,cell]
        return max[2]

                

In [69]:
class Agent:
    def __init__(self,home,counter):
        self.home = home
        self.home.occupant = self
        self.counter = counter
        self.depth = 1
    
    def randomMove(self):
        move_to = self.home.randomNeighbour()
        if move_to.empty:
            self.home.occupant = None
            self.home = move_to
            self.home.occupant = self

    def searchMove(self):
        new_pos = self.home.search_res(self.depth)
        self.home.occupant = None
        self.home = new_pos
        new_pos.occupant = self

    def move(self,direction):
        new_pos = self.home.neighbours[direction]
        if new_pos.empty:
            self.home.occupant = None
            self.home = new_pos
            new_pos.occupant = self


    

In [70]:
class Experiment:
    def __init__(self,size):        
        self.cells = []
        self.agents =[]
        self.size = size
        self.total_cells = size*size
        self.svg = SVGCanvas(size,15)
        

    def setUp(self,agents):
        self.cells = []
        self.agents = []
        self.counter = Counter()
        
        for i in range(self.size*self.size):
            n_cell = Cell((i%self.size),int(i/self.size),self.svg,self.counter)
            n_cell.resource = (i%self.size)+int(i/self.size)
            self.cells.append(n_cell)
        for cell in self.cells:
            for d_pos in[(-1,0),(0,-1),(1,0),(0,1)]:
                x=self.bounds(cell.x_pos+d_pos[0])
                y=self.bounds(cell.y_pos+d_pos[1])
                pos = y*self.size+x
                cell.addNeighbour(self.cells[pos])
        for _ in range(agents):
            #home = self.cells[1]#rnd.choice(self.cells)
            home = rnd.choice(self.cells)
            print(home.x_pos,home.y_pos)
            if home.empty:
                agent = Agent(home,self.counter)
                self.agents.append(agent)
        
    def bounds(self,i):
        if i<0:
            return self.size + i 
        if i>=self.size:
            return i-self.size
        return i
        
    def iterate(self):
        rnd.shuffle(self.agents)
        for agent in self.agents:
            agent.searchMove()
            #agent.randomMove()
            #agent.move(1)
        self.draw()
        
    def draw(self):
        self.svg.clear()
        for cell in self.cells:
            cell.draw()
        clear_output(wait=True)
        display(self.svg.getCanvas())


In [73]:
experiment = Experiment(16)
experiment.setUp(10)
for _ in range(30):
    experiment.iterate()
    time.sleep(0.25)